In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os

sys.path.insert(0, os.path.abspath(".."))
sys.path.append(os.path.abspath("../backend"))
sys.path.append(os.path.abspath(""))

In [4]:

from rdflib.plugins.stores.sparqlstore import SPARQLStore
from backend.model import *
from backend.ontology import *
from backend.datasetmatcher import *
from backend.explorative_support import *

base_path = "../data"
onto_path = f"{base_path}/hero-ontology/hereditary_clinical.ttl"


# graph = Graph().parse(onto_path, format="turtle")
# graph.bind("bto", "http://www.semanticweb.org/ontologies/2020/3/bto#")

store = SPARQLStore(
    "http://localhost:7200/repositories/dpedia",
    method="POST_FORM",
    params={"infer": False, "sameAs": False},
)
graph = Graph(store=store)

config = OntologyConfig()

ontology_manager = OntologyManager(config, graph)
dataset_manager = DatasetManager(ontology_manager)
dataset_manager.initialise(glob_path="data/datasets/ALS/**/*.csv")

# ontology_manager.load_full_graph()


-- Loading and merging datasets
0 datasets loaded


In [5]:
topic_man = TopicModelling(ontology_manager)
topic_man.initialize_topics(force=True)

Some weights of the model checkpoint at dunzhang/stella_en_400M_v5 were not used when initializing NewModel: ['new.pooler.dense.bias', 'new.pooler.dense.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
ggml_cuda_init: GGML_CUDA_FORCE_MMQ:    no
ggml_cuda_init: GGML_CUDA_FORCE_CUBLAS: no
ggml_cuda_init: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 4090, compute capability 8.9, VMM: yes
llama_load_model_from_file: using device CUDA0 (NVIDIA GeForce RTX 4090) - 21638 MiB free
llama_model_loader: loaded meta data with 27 key-value pairs and 292 tensors from /home/bkantz/.cache/h

KeyboardInterrupt: 